In [2]:
!pip install -q transformers accelerate bitsandbytes langchain langchain-community \
    langchain-huggingface faiss-cpu fastapi uvicorn pyngrok \
    youtube-transcript-api pymupdf sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 32.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 56.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.2/485.2 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 62.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.22.1 requires google-cloud-bigquery-storage>=2.0.0, which is not installed.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.47.0 which is incompatible.
google-colab 1.0.0 requires ju

In [3]:
import os
import re
import json
import torch
import threading
import traceback
from pathlib import Path
from urllib.parse import urlparse, parse_qs
from datetime import datetime
from typing import List, Dict, Optional

from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    BitsAndBytesConfig,
    pipeline
)

from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import HuggingFacePipeline

from youtube_transcript_api import YouTubeTranscriptApi

from fastapi import FastAPI, File, UploadFile, Form, HTTPException, Request
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
from pyngrok import ngrok, conf

import fitz  

2026-01-30 18:23:28.289229: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769797408.499240      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769797408.559390      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769797409.072576      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769797409.072617      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769797409.072620      55 computation_placer.cc:177] computation placer alr

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF")

In [5]:
!pip install -q huggingface_hub
from huggingface_hub import login

login(token=secret_value_0)


In [ ]:
NGROK_TOKEN = "31CaKcK72zQFZ5v6dgDeAWmtRTg_T8zD7FHZNe1WYBKrsNwN" #paste your NGROK Token from the website
API_KEY = "secret1234"
PORT = 8000
FAISS_PATH = "/kaggle/working/faiss_index"
PDF_FAISS_PATH = "/kaggle/working/pdf_faiss_index"

model_name = "meta-llama/Llama-3.1-8B-Instruct"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True,
    trust_remote_code=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)


text_gen_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1,
    max_new_tokens=1024,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    return_full_text=False
)

llm = HuggingFacePipeline(pipeline=text_gen_pipe)


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Device set to use cuda:0
/tmp/ipykernel_55/2604756406.py:48: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_gen_pipe)


In [7]:
class ConversationHistory:
    def __init__(self, max_history=10):
        self.max_history = max_history
        self.history: List[Dict] = []
    
    def add_exchange(self, user_msg: str, assistant_msg: str):
        """Add a complete exchange"""
        self.history.append({"role": "user", "content": user_msg})
        self.history.append({"role": "assistant", "content": assistant_msg})
        
        if len(self.history) > self.max_history * 2:
            self.history = self.history[-self.max_history * 2:]
    
    def get_context_string(self) -> str:
        """Get formatted history for context"""
        if not self.history:
            return ""
        
        context = "\n\nPrevious conversation:\n"
        # Use last 3 exchanges (6 messages)
        for msg in self.history[-6:]:
            role = "Human" if msg["role"] == "user" else "Assistant"
            context += f"{role}: {msg['content']}\n"
        
        return context
    
    def clear(self):
        self.history = []

In [8]:
def generate_response(prompt: str, system_prompt: str = "You are GOAT, a helpful and knowledgeable AI assistant.", 
                     max_tokens: int = 1024) -> str:
    """Generate response without storing history internally"""
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]
    
    formatted_prompt = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )
    
    output = text_gen_pipe(
        formatted_prompt,
        max_new_tokens=max_tokens,
        return_full_text=False
    )
    
    return output[0]["generated_text"].strip()

In [10]:
study_plan_prompt = PromptTemplate(
    input_variables=["subject", "level", "time_available", "goal", "timeline"],
    template="""You are GOAT, an expert educational consultant. Create a comprehensive study plan.

Subject: {subject}
Level: {level}
Time Available: {time_available} hours/week
Goal: {goal}
Timeline: {timeline}  # Timeline will be a natural description like '1 month', '3 months', '6 months', or '1 year'

GUIDELINES:
- Based on {level} and {goal}
- Do NOT convert the timeline into weeks; use the timeline as a guide for pacing.
- Divide learning into logical phases: Foundation → Intermediate → Mastery.
- Suggest milestones and goals for each phase relative to the timeline.
- Suggest weekly/daily routines without assigning fixed week numbers.
- Make the plan concise, actionable, and structured.

WEEKLY BREAKDOWN
- Suggest topics and focus areas per week without using hard-coded week numbers

DAILY SCHEDULE
- How to distribute {time_available} hours/week
- Monday-Friday routine
- Weekend activities

MILESTONES
- Define key achievements for each phase (Foundation, Intermediate, Mastery)
- Use the timeline to scale milestones appropriately

LEARNING RESOURCES
- Top 3 online courses
- Top 3 books
- Top 3 practice platforms

Keep sections clearly separated with single line breaks."""
)


study_plan_chain = study_plan_prompt | llm | StrOutputParser()

roadmap_prompt = PromptTemplate(
    input_variables=["subject", "level", "goal"],
    template="""You are GOAT, a learning path architect.

Create a structured and practical learning roadmap.

Subject: {subject}
Starting Level: {level}
Goal: {goal}

GUIDELINES:
- Based on {level} and {goal}
- Divide the learning journey into clear phases:
  Foundation → Intermediate → Mastery
- Focus on progression of skills, not duration
- Do NOT mention specific weeks or months
- Keep the roadmap flexible and adaptable
- Emphasize practical skills and real-world application

PHASE 1 – FOUNDATION
- Core concepts to understand
- Fundamental skills to build
- Beginner-level practice ideas

PHASE 2 – INTERMEDIATE
- More advanced topics
- Hands-on projects
- Skills needed for real use cases

PHASE 3 – MASTERY
- Expert-level concepts
- Real-world and portfolio projects
- How this phase moves the learner closer to the goal

Use concise bullet points.
Keep the roadmap clear, actionable, and complete.
Use single line breaks between sections."""
)

roadmap_chain = roadmap_prompt | llm | StrOutputParser()

resources_prompt = PromptTemplate(
    input_variables=["subject", "goal"],
    template="""You are GOAT, a learning resource curator.

Recommend the best resources for a learner with the following goal:

Subject: {subject}
Goal: {goal}

GUIDELINES:
- Tailor resources to the learner's timeline and goal
- Prioritize high-quality and actionable materials
- Include specific recommendations for each type

RECOMMENDATIONS:

ONLINE COURSES (Top 3)
1. Course name - Platform - Brief description
2. 
3. 

BOOKS (Top 3)
1. Title - Author - Why valuable
2.
3.

PRACTICE PLATFORMS (Top 3)
1. Platform - What to practice
2.
3.

COMMUNITIES (Top 2)
1. Community - How it helps
2.

Keep the recommendations structured and concise.
Use single line breaks between sections."""
)

resources_chain = resources_prompt | llm | StrOutputParser()


In [11]:
youtube_summary_prompt = PromptTemplate(
    input_variables=["transcript"],
    template="""You are GOAT. Summarize this video transcript comprehensively.

Transcript:
{transcript}

Create a structured summary with:

MAIN TOPIC
- Brief description

KEY POINTS
- Point 1
- Point 2
- Point 3
- Point 4
- Point 5

IMPORTANT DETAILS
- Detail 1
- Detail 2
- Detail 3

CONCLUSIONS
- Final takeaway

Keep it clear and well-organized. Use single line breaks between sections."""
)

youtube_summary_chain = youtube_summary_prompt | llm | StrOutputParser()

youtube_chunk_prompt = PromptTemplate(
    input_variables=["chunk"],
    template="""Summarize this video segment in 3-4 concise bullet points:

{chunk}"""
)

youtube_chunk_chain = youtube_chunk_prompt | llm | StrOutputParser()

youtube_reduce_prompt = PromptTemplate(
    input_variables=["summaries"],
    template="""Combine these segment summaries into one coherent report:

{summaries}

Format:

MAIN TOPIC
- Brief description

KEY POINTS
- Point 1
- Point 2
- Point 3
- Point 4
- Point 5

IMPORTANT DETAILS
- Detail 1
- Detail 2
- Detail 3

CONCLUSIONS
- Final takeaway

Keep it concise and well-structured. Use single line breaks between sections."""
)

youtube_reduce_chain = youtube_reduce_prompt | llm | StrOutputParser()


In [12]:
def extract_video_id(url: str) -> str:
    """Extract YouTube video ID from URL"""
    parsed = urlparse(url)
    if parsed.hostname in ['youtu.be']:
        return parsed.path[1:]
    if parsed.hostname in ['www.youtube.com', 'youtube.com']:
        if parsed.path == '/watch':
            return parse_qs(parsed.query)['v'][0]
        if parsed.path.startswith(('/embed/', '/v/')):
            return parsed.path.split('/')[2]
    raise ValueError(f"Could not extract video ID from URL: {url}")

def get_transcript(video_id: str) -> str:
    """Fetch YouTube transcript"""
    try:
        api = YouTubeTranscriptApi()
        fetched = api.fetch(video_id)  # returns a FetchedTranscript
        text = "\n".join(snippet.text for snippet in fetched)
        return text
    except Exception as e:
        return f"Error: {str(e)}"

def extract_text_from_pdf(pdf_bytes: bytes) -> str:
    """Extract text from PDF bytes"""
    try:
        doc = fitz.open(stream=pdf_bytes, filetype="pdf")
        text = ""
        for page in doc:
            text += page.get_text()
        doc.close()
        
        if not text.strip():
            return "Error: PDF appears to be empty or contains only images"
        
        return text
    except Exception as e:
        return f"Error extracting PDF text: {str(e)}"


def setup_rag(text_content: str, index_path: str):
    """Build and persist FAISS vector store"""
    docs = [Document(page_content=text_content, metadata={"source": "document"})]
    
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=100
    )
    chunks = splitter.split_documents(docs)
    
    embedding = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    
    vectordb = FAISS.from_documents(chunks, embedding)
    vectordb.save_local(index_path)
    
    return vectordb

def load_rag(index_path: str):
    """Load existing FAISS index"""
    if os.path.exists(index_path):
        return FAISS.load_local(
            index_path,
            HuggingFaceEmbeddings(
                model_name="sentence-transformers/all-MiniLM-L6-v2"
            ),
            allow_dangerous_deserialization=True
        )
    return None

def chunk_transcript(transcript: str, words_per_chunk: int = 5000):
    """Split transcript into larger chunks for fewer API calls"""
    words = transcript.split()
    for i in range(0, len(words), words_per_chunk):
        yield " ".join(words[i:i + words_per_chunk])

def process_summarization(full_transcript: str) -> str:
    """Process video summarization using LangChain chains"""
    clean_text = re.sub(r'\[.*?\]', '', full_transcript)
    clean_text = re.sub(r'\s+', ' ', clean_text).strip()
    
    chunks = list(chunk_transcript(clean_text, words_per_chunk=5000)) 
    
    print(f"🔄 Summarizing {len(chunks)} segments...")
    
    if len(chunks) <= 2:
        print("📝 Short video - using direct summary chain...")
        summary = youtube_summary_chain.invoke({"transcript": clean_text[:15000]})
        return summary
    
    chunk_summaries = []
    
    for i, chunk in enumerate(chunks):
        print(f"📝 Processing segment {i+1}/{len(chunks)} with chain...")
        summary = youtube_chunk_chain.invoke({"chunk": chunk})
        chunk_summaries.append(summary)
    
    print("🏁 Synthesizing final report with reduce chain...")
    combined_text = "\n\n".join(chunk_summaries)
    
    final_summary = youtube_reduce_chain.invoke({"summaries": combined_text})
    return final_summary

In [13]:
class AppState:
    def __init__(self):
        # YouTube
        self.youtube_vectordb = None
        self.youtube_transcript = None
        self.youtube_history = ConversationHistory(max_history=10)
        
        # PDF
        self.pdf_vectordb = None
        self.pdf_text = None
        self.pdf_history = ConversationHistory(max_history=10)
        
        # General Chat
        self.general_history = ConversationHistory(max_history=15)

state = AppState()


app = FastAPI(title="AI Assistant Backend")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get("/")
async def health():
    return {
        "status": "online",
        "model": "Llama-3.1-8B-Instruct",
        "features": ["study_plan", "youtube", "pdf_qa", "chat"],
        "timestamp": datetime.now().isoformat()
    }


@app.post("/study-plan/generate")
async def create_study_plan(
    request: Request,
    subject: str = Form(...),
    level: str = Form(...),
    time_available: str = Form(...),
    goal: str = Form(...),
    timeline: str = Form(...)
):
    if request.headers.get("authorization") != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401)
    
    try:
        start_time = datetime.now()
        
        temp_pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
            max_new_tokens=4096, 
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            return_full_text=False
        )
        temp_llm = HuggingFacePipeline(pipeline=temp_pipe)
        temp_chain = study_plan_prompt | temp_llm | StrOutputParser()
        
        study_plan = temp_chain.invoke({
            "subject": subject,
            "level": level,
            "time_available": time_available,
            "goal": goal,
            "timeline": timeline
        })
        
        study_plan = re.sub(r'\n{3,}', '\n\n', study_plan)
        
        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds()
        
        return {
            "success": True,
            "study_plan": study_plan,
            "timestamp": end_time.isoformat(),
            "generation_time": f"{duration:.2f} seconds"
        }
    except Exception as e:
        print(traceback.format_exc())
        return {"success": False, "error": str(e)}

@app.post("/study-plan/roadmap")
async def create_roadmap(
    request: Request,
    subject: str = Form(...),
    level: str = Form(...),
    goal: str = Form(...)
):
    if request.headers.get("authorization") != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401)
    
    try:
        start_time = datetime.now()
        
        temp_pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
            max_new_tokens=4096,  
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            return_full_text=False
        )
        temp_llm = HuggingFacePipeline(pipeline=temp_pipe)
        temp_chain = roadmap_prompt | temp_llm | StrOutputParser()
        
        roadmap = temp_chain.invoke({
            "subject": subject,
            "level": level,
            "goal": goal
        })
        
        roadmap = re.sub(r'\n{3,}', '\n\n', roadmap)
        
        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds()
        
        return {
            "success": True,
            "roadmap": roadmap,
            "timestamp": end_time.isoformat(),
            "generation_time": f"{duration:.2f} seconds"
        }
    except Exception as e:
        print(traceback.format_exc())
        return {"success": False, "error": str(e)}

@app.post("/study-plan/resources")
async def get_resources(
    request: Request,
    subject: str = Form(...),
    goal: str = Form(...)
):
    if request.headers.get("authorization") != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401)
    
    try:
        start_time = datetime.now()
        
        resources = resources_chain.invoke({
            "subject": subject,
            "goal": goal
        })
        
        resources = re.sub(r'\n{3,}', '\n\n', resources)
        
        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds()
        
        return {
            "success": True,
            "resources": resources,
            "timestamp": end_time.isoformat(),
            "generation_time": f"{duration:.2f} seconds"
        }
    except Exception as e:
        print(traceback.format_exc())
        return {"success": False, "error": str(e)}

@app.post("/youtube/process")
async def process_youtube(request: Request, url: str = Form(...), transcript: str = Form(...)):
    """UPDATED: Now accepts transcript from frontend instead of fetching"""
    if request.headers.get("authorization") != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401)
    
    try:
        if not transcript or transcript.startswith("Error"):
            return {"success": False, "error": "Invalid transcript provided"}
        
        state.youtube_transcript = transcript
        state.youtube_vectordb = setup_rag(transcript, FAISS_PATH)
        state.youtube_history.clear()
        
        return {
            "success": True,
            "message": "Video processed successfully",
            "transcript_length": len(transcript),
            "timestamp": datetime.now().isoformat()
        }
    except Exception as e:
        print(traceback.format_exc())
        return {"success": False, "error": str(e)}

@app.post("/youtube/summarize")
async def summarize_youtube(request: Request):
    if request.headers.get("authorization") != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401)
    
    if not state.youtube_transcript:
        return {"success": False, "error": "No video processed."}
    
    try:
        start_time = datetime.now()
        summary = process_summarization(state.youtube_transcript)
        
        summary = re.sub(r'\n{3,}', '\n\n', summary)
        
        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds()
        
        return {
            "success": True,
            "summary": summary,
            "timestamp": end_time.isoformat(),
            "generation_time": f"{duration:.2f} seconds"
        }
    except Exception as e:
        print(traceback.format_exc())
        return {"success": False, "error": str(e)}

@app.post("/youtube/chat")
async def chat_youtube(request: Request, question: str = Form(...)):
    if request.headers.get("authorization") != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401)
    
    if not state.youtube_vectordb:
        return {"success": False, "error": "No video processed."}
    
    try:
        start_time = datetime.now()
        
        docs = state.youtube_vectordb.similarity_search(question, k=3)
        context = "\n\n".join(d.page_content for d in docs)
        
        history_context = state.youtube_history.get_context_string()
        
        full_prompt = f"""Context from video:
{context}
{history_context}

Current question: {question}

Answer based on the video content. Be specific and reference relevant information."""
        
        answer = generate_response(full_prompt, max_tokens=800)
        
        state.youtube_history.add_exchange(question, answer)
        
        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds()
        
        return {
            "success": True,
            "answer": answer,
            "timestamp": end_time.isoformat(),
            "generation_time": f"{duration:.2f} seconds"
        }
    except Exception as e:
        print(traceback.format_exc())
        return {"success": False, "error": str(e)}

@app.post("/youtube/clear-history")
async def clear_youtube_history(request: Request):
    if request.headers.get("authorization") != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401)
    state.youtube_history.clear()
    return {"success": True, "message": "History cleared"}

@app.post("/pdf/upload")
async def upload_pdf(request: Request, file: UploadFile = File(...)):
    if request.headers.get("authorization") != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401)
    
    try:
        pdf_bytes = await file.read()
        text = extract_text_from_pdf(pdf_bytes)
        
        if text.startswith("Error"):
            return {"success": False, "error": text}
        
        state.pdf_text = text
        state.pdf_vectordb = setup_rag(text, PDF_FAISS_PATH)
        state.pdf_history.clear()
        
        return {
            "success": True,
            "message": "PDF processed successfully",
            "text_length": len(text),
            "timestamp": datetime.now().isoformat()
        }
    except Exception as e:
        print(traceback.format_exc())
        return {"success": False, "error": str(e)}

@app.post("/pdf/ask")
async def ask_pdf(request: Request, question: str = Form(...)):
    if request.headers.get("authorization") != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401)
    
    if not state.pdf_vectordb:
        return {"success": False, "error": "No PDF uploaded."}
    
    try:
        start_time = datetime.now()
        
        docs = state.pdf_vectordb.similarity_search(question, k=4)
        context = "\n\n".join(d.page_content for d in docs)
        
        history_context = state.pdf_history.get_context_string()
        
        full_prompt = f"""Context from PDF:
{context}
{history_context}

Current question: {question}

Answer based on the PDF content. Be accurate and reference specific information."""
        
        answer = generate_response(full_prompt, max_tokens=800)
        
        state.pdf_history.add_exchange(question, answer)
        
        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds()
        
        return {
            "success": True,
            "answer": answer,
            "timestamp": end_time.isoformat(),
            "generation_time": f"{duration:.2f} seconds"
        }
    except Exception as e:
        print(traceback.format_exc())
        return {"success": False, "error": str(e)}

@app.post("/pdf/clear-history")
async def clear_pdf_history(request: Request):
    if request.headers.get("authorization") != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401)
    state.pdf_history.clear()
    return {"success": True, "message": "History cleared"}

@app.post("/chat")
async def general_chat(request: Request, message: str = Form(...)):
    if request.headers.get("authorization") != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401)
    
    try:
        start_time = datetime.now()
        
        history_context = state.general_history.get_context_string()
        
        full_prompt = f"""{history_context}

Current message: {message}

Respond helpfully and naturally."""
        
        response = generate_response(
            full_prompt,
            system_prompt="You are GOAT, a helpful and friendly AI assistant. Be conversational and informative.",
            max_tokens=800
        )
        
        state.general_history.add_exchange(message, response)
        
        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds()
        
        return {
            "success": True,
            "response": response,
            "timestamp": end_time.isoformat(),
            "generation_time": f"{duration:.2f} seconds"
        }
    except Exception as e:
        print(traceback.format_exc())
        return {"success": False, "error": str(e)}

@app.post("/chat/clear-history")
async def clear_chat_history(request: Request):
    if request.headers.get("authorization") != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401)
    state.general_history.clear()
    return {"success": True, "message": "History cleared"}


In [14]:
print("\n🚀 Starting server with NGROK...")

conf.get_default().auth_token = NGROK_TOKEN

try:
    ngrok.kill()
except:
    pass

public_url = ngrok.connect(PORT).public_url
print(f"\n✅ BACKEND URL: {public_url}")
print(f"🔑 API KEY: {API_KEY}\n")

def run():
    uvicorn.run(app, host="0.0.0.0", port=PORT, log_level="info")

threading.Thread(target=run, daemon=True).start()


🚀 Starting server with NGROK...
                                                                                                    
✅ BACKEND URL: https://ungoaded-subdorsally-bethanie.ngrok-free.dev
🔑 API KEY: secret1234



INFO:     Started server process [55]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
Device set to use cuda:0


INFO:     156.202.70.116:0 - "POST /study-plan/roadmap HTTP/1.1" 200 OK
INFO:     156.202.70.116:0 - "GET / HTTP/1.1" 200 OK
INFO:     156.202.70.116:0 - "GET / HTTP/1.1" 200 OK
INFO:     156.202.70.116:0 - "GET / HTTP/1.1" 200 OK
INFO:     156.202.70.116:0 - "GET / HTTP/1.1" 200 OK


Device set to use cuda:0


INFO:     156.202.70.116:0 - "POST /study-plan/generate HTTP/1.1" 200 OK


...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

INFO:     156.202.70.116:0 - "POST /youtube/process HTTP/1.1" 200 OK
🔄 Summarizing 1 segments...
📝 Short video - using direct summary chain...
INFO:     156.202.70.116:0 - "POST /youtube/summarize HTTP/1.1" 200 OK
INFO:     156.202.70.116:0 - "POST /youtube/chat HTTP/1.1" 200 OK
INFO:     156.202.70.116:0 - "POST /youtube/chat HTTP/1.1" 200 OK
INFO:     156.202.70.116:0 - "POST /pdf/upload HTTP/1.1" 200 OK
INFO:     156.202.70.116:0 - "POST /pdf/ask HTTP/1.1" 200 OK
INFO:     156.202.70.116:0 - "POST /pdf/ask HTTP/1.1" 200 OK
INFO:     156.202.70.116:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     156.202.70.116:0 - "POST /chat HTTP/1.1" 200 OK
